# Bone Fracture Detection using YOLOv8

This Jupyter Notebook explores the application of deep learning for bone fracture detection using a comprehensive X-ray image dataset.  The dataset is specifically designed for computer vision projects and aims to facilitate the development and evaluation of automated bone fracture detection algorithms.

## About the Dataset

The dataset encompasses X-ray images categorized into several classes, each representing a specific type of bone fracture within the upper extremities. These classes include:

*   Elbow Positive
*   Fingers Positive
*   Forearm Fracture
*   Humerus Fracture
*   Shoulder Fracture
*   Wrist Positive

Each image is annotated with either bounding boxes or pixel-level segmentation masks, precisely indicating the location and extent of the detected fracture. These annotations are crucial for training and evaluating bone fracture detection algorithms, particularly object detection models.

This dataset provides a valuable resource for researchers and developers working on automated fracture detection. Its diverse range of fracture classes enables the training of robust models capable of accurately identifying fractures in various regions of the upper extremities. The ultimate goal of this dataset is to accelerate the development of computer vision solutions for automated fracture detection, thereby contributing to advancements in medical diagnostics and improved patient care.

**When using this dataset for your research, please cite it using the following DOI:** 10.13140/RG.2.2.14400.34569

**You can also find the dataset on ResearchGate:** [https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project](https://www.researchgate.net/publication/382268240_Bone_Fracture_Detection_Computer_Vision_Project)

## Imports

In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU'), tf.__version__

2025-02-12 10:26:10.536720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739355970.549342   46226 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739355970.552930   46226 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 10:26:10.565963: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


([PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], '2.18.0')

In [2]:

from dotenv import load_dotenv
import keras
from pathlib import Path
import cv2

In [3]:
from utils.prepare_dataset import PrepareDataset

## Download datasets

In [4]:
# Loading kaggle keys
load_dotenv()

True

In [5]:
%load_ext dotenv
%dotenv

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from hydra import initialize, compose

# https://gist.github.com/bdsaglam/586704a98336a0cf0a65a6e7c247d248

with initialize(version_base=None, config_path="conf"):
    cfg = compose(config_name="config")
    print(cfg.DATASET_DIRS.TRAIN_DIR)

datasets/BoneFractureYolo8/train/


## Loading Images

In [8]:
CLASS_NAMES = ['elbow positive',
               'fingers positive',
               'forearm fracture',
               'humerus fracture',
               'humerus',
               'shoulder fracture',
               'wrist positive']

class_mapping = dict(zip(range(len(CLASS_NAMES)), CLASS_NAMES))
class_mapping

{0: 'elbow positive',
 1: 'fingers positive',
 2: 'forearm fracture',
 3: 'humerus fracture',
 4: 'humerus',
 5: 'shoulder fracture',
 6: 'wrist positive'}

In [9]:
TRAIN_DIR = Path(cfg.DATASET_DIRS.TRAIN_DIR)
VALIDATION_DIR = Path(cfg.DATASET_DIRS.VALIDATION_DIR)
TEST_DIR = Path(cfg.DATASET_DIRS.TEST_DIR)

TRAIN_IMAGE_DIR = TRAIN_DIR / 'images'
TRAIN_LABELS_DIR = TRAIN_DIR / 'labels'

VALID_IMAGE_DIR = VALIDATION_DIR/'images'
VALID_LABELS_DIR = VALIDATION_DIR/'labels'

TEST_IMAGE = TEST_DIR/'images'
TEST_LABELS = TEST_DIR/'labels'

IMG_SIZE = cfg.TRAIN.IMG_SIZE
BATCH_SIZE = cfg.TRAIN.BATCH_SIZE

### Training and Validation Dataset setup

In [10]:
def load_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    resize_img = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    scaled_img = resize_img / 255.
    return tf.cast(scaled_img, tf.float32)


def load_dataset(image_path, classes, bbox):
    # Read Image
    image = load_image(image_path)
    # return image, tf.one_hot(classes, len(CLASS_NAMES)), bbox
    return image, {
    'classification': tf.one_hot(classes, len(CLASS_NAMES)),
    'bounding_box': bbox
}

#### Train Datasets setup

In [11]:
prepare_train_datasets = PrepareDataset(image_dir=TRAIN_IMAGE_DIR,
                                   label_dir=TRAIN_LABELS_DIR,
                                   dst_img_size=(IMG_SIZE, IMG_SIZE))
image_paths, class_ids, bboxes = prepare_train_datasets.get_dataset()
len(image_paths), len(class_ids), len(bboxes)

0it [00:00, ?it/s]

(2088, 2088, 2088)

In [12]:
class_ids[:10]

[1, 6, 0, 6, 2, 0, 0, 0, 2, 6]

In [13]:
bboxes[:10]

[array([[0.4404683 , 0.31309316, 0.7659444 , 0.55647546]], dtype=float32),
 array([[0.2908256 , 0.38371962, 0.41078433, 0.51536673]], dtype=float32),
 array([[0.4224264 , 0.4298628 , 0.60367703, 0.54083246]], dtype=float32),
 array([[0.41856024, 0.4094788 , 0.6055509 , 0.4855385 ]], dtype=float32),
 array([[0.365293  , 0.396875  , 0.5938265 , 0.55958796]], dtype=float32),
 array([[0.72946703, 0.3694884 , 0.8371946 , 0.49267924]], dtype=float32),
 array([[0.5997687 , 0.3987722 , 0.6947129 , 0.51465213]], dtype=float32),
 array([[0.55960584, 0.3642187 , 0.7474956 , 0.49238095]], dtype=float32),
 array([[0.5768778 , 0.60538375, 0.6844229 , 0.711113  ]], dtype=float32),
 array([[0.38905194, 0.4906167 , 0.7111077 , 0.591931  ]], dtype=float32)]

In [14]:
train_datasets = tf.data.Dataset.from_tensor_slices((image_paths, 
                                                     class_ids, 
                                                     bboxes))

I0000 00:00:1739355974.469815   46226 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7224 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6


In [15]:
train_ds = train_datasets.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)\
                     .shuffle(BATCH_SIZE * 4)\
                     .repeat()\
                     .batch(BATCH_SIZE, drop_remainder=True)\
                     .prefetch(tf.data.AUTOTUNE)


In [16]:
for batch in train_ds.take(1):
    print(batch)

(<tf.Tensor: shape=(16, 224, 224, 3), dtype=float32, numpy=
array([[[[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]],

        [[0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
         ...,
         [0.0000000e+00, 0.0000000e+00, 0.0000000e+00]

2025-02-12 10:26:14.770620: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Validation datasets setup

In [17]:
prepare_valid_datasets = PrepareDataset(image_dir=VALID_IMAGE_DIR,
                                label_dir=VALID_LABELS_DIR)
valid_image_paths, valid_class_ids, valid_bboxes = prepare_valid_datasets.get_dataset()
len(valid_image_paths), len(valid_class_ids), len(valid_bboxes)

0it [00:00, ?it/s]

[ERROR] - too many indices for array: array is 1-dimensional, but 2 were indexed in file datasets/BoneFractureYolo8/valid/labels/image2_1224_png.rf.7c20f6876b3efc39ea916cda7db4c00c.txt on line: 4 0.4485848624751253 0.20156250149011612 0.6407782735648366 0.220426963092181 0.6237999317132228 0.41892209685605486

[ERROR] - too many indices for array: array is 1-dimensional, but 2 were indexed in file datasets/BoneFractureYolo8/valid/labels/image2_610_png.rf.c0995510828ea9894c4ffdea39630aca.txt on line: 2 0.31952376592726933 0.5218750014901161 0.41358553521531183 0.5267658674366184 0.3564521642623 0.5908470055359224
[ERROR] - too many indices for array: array is 1-dimensional, but 2 were indexed in file datasets/BoneFractureYolo8/valid/labels/image1_1031_png.rf.0f84f2c23a4cd720a80e74e445be6466.txt on line: 0 0.5447003764490927 0.3656250014901161 0.6262460103430545 0.33921142160403556 0.622560042654864 0.3376484917375452


(201, 201, 201)

In [18]:
val_datasets = tf.data.Dataset.from_tensor_slices((valid_image_paths,
                                               valid_class_ids,
                                               valid_bboxes))
val_ds = val_datasets.map(load_dataset, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.shuffle(BATCH_SIZE * 4) \
                .repeat()\
                .batch(BATCH_SIZE, drop_remainder=True)\
                .prefetch(tf.data.AUTOTUNE)

## Define DenseNet121 Model

### Define densenet121 Preprocessor

In [19]:
def densenet121_preprocessing(inputs)->keras.Model:
    return tf.keras.applications.densenet.preprocess_input(inputs)

### Define DenseNet121 as a Feature Extractor

In [20]:
def feature_extractor(inputs)-> keras.Model:
    # Create a DenseNet121 model object
    densenet121 = keras.applications.DenseNet121(
        include_top = False, 
        weights = "imagenet",
        input_shape = (IMG_SIZE, IMG_SIZE, 3),
        input_tensor=inputs
    )
    
    densenet121.trainable = False

    feature_extractor = densenet121.output
    return feature_extractor

### Define Dense Layers

In [43]:
def dense_layers(features)->keras.Layer:
    x = tf.keras.layers.GlobalAveragePooling2D()(features)

    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
    x = tf.keras.layers.Dense(units=1024, activation='relu')(x)
    x = tf.keras.layers.Dense(units=512, activation='relu')(x)

    return x

### Define Bounding Box Regression

In [44]:

def bounding_box_regression(x)->keras.Layer:

    # Dense layer named `bounding_box`
    bounding_box_regression_output = tf.keras.layers.Dense(units=4, name='bounding_box')(x)

    return bounding_box_regression_output

### Define Classifier Layer

In [45]:
def classifer(inputs)->keras.Model:
    return tf.keras.layers.Dense(units=len(CLASS_NAMES), activation='softmax',  name = 'classification')(inputs)

### Final Model

In [56]:
def final_model(inputs)->keras.Model:
    preprocessed_inputs = densenet121_preprocessing(inputs)
    
    _feature_extractor = feature_extractor(preprocessed_inputs)
    
    dense_output = dense_layers(_feature_extractor)

    bounding_box_regression_output = bounding_box_regression(dense_output)

    classification_output = classifer(dense_output)

    return tf.keras.Model(inputs=inputs, 
                          outputs=[classification_output, bounding_box_regression_output])



### Define  Callbacks

In [57]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['classification_accuracy'] >= 0.90:
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

# TODO add more call backs

In [58]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [59]:
inputs = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

densenet121_model = final_model(inputs)

In [60]:

densenet121_model.compile(
    optimizer=optimizer,
    loss={'classification': 'categorical_crossentropy', 
          'bounding_box': 'mean_squared_error'},
    metrics={'classification': 'accuracy', 
             'bounding_box': 'mse'})

In [61]:
densenet121_model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ true_divide_6       │ (None, 224, 224,  │          0 │ input_layer_3[0]… │
│ (TrueDivide)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 224, 224,  │          0 │ true_divide_6[0]… │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ true_divide_7       │ (None, 224, 224,  │          0 │ add_3[0][0]       │
│ (TrueDivide)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_6    │ (None, 230, 230,  │          0 │ true_divide_7[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,408 │ zero_padding2d_6… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_7    │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 56, 56,    │          0 │ zero_padding2d_7… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 56, 56,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 56, 56,    │          0 │ pool1[0][0],    

 Total params: 9,667,147 (36.88 MB)

 Trainable params: 2,629,643 (10.03 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

## Train and Validate the model

In [62]:
EPOCHS = 50

In [63]:
len(train_datasets)

2088

In [64]:
import math
# Get the length of the training set
length_of_training_dataset = len(train_datasets)

# Get the length of the validation set
length_of_validation_dataset = len(val_datasets)

# Get the steps per epoch (may be a few lines of code)
steps_per_epoch = math.ceil(length_of_training_dataset/BATCH_SIZE)

# get the validation steps (per epoch) (may be a few lines of code)
validation_steps = math.ceil(length_of_validation_dataset/BATCH_SIZE)


In [ ]:
history = densenet121_model.fit(
    train_ds,
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS,
    validation_data=val_ds,
    validation_steps=validation_steps,
    callbacks=[myCallback()],
)

Epoch 1/50
130/131 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - bounding_box_loss: 0.0500 - bounding_box_mse: 0.0500 - classification_accuracy: 0.2315 - classification_loss: 1.8228 - loss: 1.8728